In [ ]:
from ML_utils import ML_utils
from gensim.models import word2vec
import shutil
import os
import numpy as np
import pandas as pd
import re
import logging
import sys

# train wordvec

In [ ]:
file_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\merge_wiki.txt"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\merge_wiki_tw.txt"
ML_utils.Opencc_file(file_path,save_path,replace_old=True)

In [ ]:
userdict_path = './jieba_dict/ptt_userdict.txt'
jieba_dict = 'jieba_dict/jieba_merge.txt'
dir_path = r"./word2vec_model"
ML_utils.CallF_DirFile_save(dir_path, ML_utils.Jieba_file_segmentation, 
                            replace_old=False,regular=True,file_filter_=None,
                            dict_path=jieba_dict,load_userdict_path=userdict_path,HMM=False)

In [ ]:
dir_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
ML_utils.Word2vec_train(file_path=None,dir_path=dir_path,save_path=save_path,model_min_count=5)

In [ ]:
#test
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
model = word2vec.Word2Vec.load(save_path)
model.wv.most_similar(positive='台灣',topn=20)

# word2vec add token

In [ ]:
#add token
model_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
train_file_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\jieba_dict\token\token.txt"
ML_utils.add_word2vec_word(model_path,save_path,train_file_path=train_file_path)

In [ ]:
#test
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\word2vec_model\model_wordvec1"
model = word2vec.Word2Vec.load(save_path)
model.wv.most_similar(positive='__SOS.Token__',topn=20)

# data add token

In [4]:
data_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt.txt"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token.txt"
ML_utils.data_add_token_SOS_EOS(data_path,save_path)

# data jieba segmentation

In [ ]:
data_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token.txt"
save_path = r"D:\Backup\ml_data\GitHub\chat-robot_seq2seq\data\merge_ptt_token_jieba.txt"
load_userdict_path = './jieba_dict/ptt_userdict.txt'
dict_path = 'jieba_dict/jieba_merge.txt'
ML_utils.Jieba_file_segmentation(data_path,save_path,
                                 dict_path=dict_path,load_userdict_path=load_userdict_path,HMM=False)